In [1]:
!pip install nltk
!pip install contractions
!pip install vaderSentiment
import nltk
nltk.download('punkt')
nltk.download('stopwords')

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.
[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>


False

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

import os
import pickle
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
import numpy as np
import pandas as pd
import contractions
import re
import string

In [4]:
file_name="cell_phones"


In [5]:
df=pd.read_csv(file_name+".csv")

In [6]:
df

,UserID,ProductID,Review,Overall Rating,Review Time
0,A24E3SXTC62LJI,7508492919,Looks even better in person. Be careful to not...,5.0,1407110400
1,A269FLZCB4GIPV,7508492919,When you don't want to spend a whole lot of ca...,5.0,1392163200
2,AB6CHQWHZW4TV,7508492919,"so the case came on time, i love the design. I...",3.0,1391817600
3,A1M117A53LEI8,7508492919,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,2.0,1391472000
4,A272DUT8M88ZS8,7508492919,"I liked it because it was cute, but the studs ...",4.0,1391385600
...,...,...,...,...,...
1127667,A1QWMCG1FNEP3A,B01HJC7N4C,Good for viewing. But doesn't have a button or...,4.0,1482364800
1127668,A3FOBEJ9UVUTR3,B01HJC7N4C,I was given the Rockrok 3D VR Glasses Headset ...,5.0,1468540800
1127669,AMUEAMKB4E33M,B01HJC7N4C,Super Fun! The RockRoc 3d vr headset is waaaay...,5.0,1468454400
1127670,A2EV91MMOJ3IL4,B01HJC7N4C,Love it!\n\nI've had other VR glasses which al...,5.0,1468368000


In [7]:
df.describe()

,Overall Rating,Review Time
count,1.127672e+06,1.127672e+06
mean,4.221138e+00,1.440386e+09
std,1.232042e+00,4.569655e+07
min,1.000000e+00,1.035331e+09
25%,4.000000e+00,1.416355e+09
50%,5.000000e+00,1.444349e+09
75%,5.000000e+00,1.470442e+09
max,5.000000e+00,1.538438e+09


Note that the rating is highly skewed, mean rating ~4.7

In [8]:
def convertToLower(text):
    text = contractions.fix(str(text))
    text=str(text).lower()
    return text

In [9]:
df["Review"]=df["Review"].apply(lambda x: convertToLower(x))

In [10]:
df["Review"]

0          looks even better in person. be careful to not...
1          when you do not want to spend a whole lot of c...
2          so the case came on time, i love the design. i...
3          do not care for it.  gave it as a gift and the...
4          i liked it because it was cute, but the studs ...
                                 ...                        
1127667    good for viewing. but does not have a button o...
1127668    i was given the rockrok 3d vr glasses headset ...
1127669    super fun! the rockroc 3d vr headset is waaaay...
1127670    love it!\n\ni have had other vr glasses which ...
1127671                                            i love it
Name: Review, Length: 1127672, dtype: object

In [11]:
def removeURLS(text):
    text = re.sub(r"((https?:\/\/)|(www\.)){1}[a-zA-Z0-9@:%._\+~#=/-]+"," ",text)
    return text

In [12]:
def removeHTMLTags(text):

    text = re.sub(r"<\/?\s*(html|base|head|link|meta|style|title|body|address|article|aside|footer|header|h1|h2|h3|h4|h5|h6|main|nav|section|blockquote|dd|div|dl|dt|figcaption|figure|hr|li|menu|ol|ul|p|pre|a|abbr|b|bdl|bdo|br|cite|code|data|dfn|em|i|kbd|mark|q|rp|rt|ruby|s|samp|small|span|strong|sub|sup|time|u|var|wbr|area|audio|img|map|track|video|embed|iframe|object|picture|portal|source|svg|math|canvas|noscript|script|del|ins|caption|col|colgroup|table|tbody|td|tfoot|th|thead|tr|button|datalist|fieldset|form|input|label|legend|meter|optgroup|option|output|progress|select|textarea|details|diolog|summary|slot|template)\s*([A-Za-z\s\-]*\s*\=[\.\'\"_A-Z0-9\sa-z\/\,\-\=\:\;]*)?\/?\s*>"," ",text)
    return text

In [ ]:
df["Review"]=df["Review"].apply(lambda x: removeURLS(x))

In [ ]:
df["Review"]=df["Review"].apply(lambda x: removeHTMLTags(x))

In [ ]:
def removePunctuations(text):
    list_punc=list(string.punctuation)
    for i in range(len(text)):
        if text[i] in list_punc:
            listText=list(text)
            listText[i]=' '
            text = ''.join(listText)
    return text

In [ ]:
df["Review"]=df["Review"].apply(lambda x: removePunctuations(x))

In [ ]:
df["Review"]

In [ ]:
def removeExtraWhiteSpaces(text):
    text = re.sub("\s+"," ",text)
    return text

In [ ]:
df["Review"]=df["Review"].apply(lambda x: removeExtraWhiteSpaces(x))

In [ ]:
STOP_WORDS=stopwords.words('english')

STOP_WORDS.append("a")
STOP_WORDS.append("an")
STOP_WORDS.append("the")

In [ ]:
# def removeStopWords(tokens):
#     new_tokens = [token for token in tokens if token not in STOP_WORDS]
#     return new_tokens

In [ ]:
def tokenize_text(text):
    tokens=word_tokenize(text)
    return tokens

In [ ]:
df["Review"]=df["Review"].apply(lambda x: tokenize_text(x))

In [ ]:
df["Review"]

In [ ]:
# df["Review"]=df["Review"].apply(lambda x: removeStopWords(x))

In [ ]:
str(df["Review"][0])

In [ ]:
# stemmer = PorterStemmer()

In [ ]:
# def stemText(tokens):
#     stemTokens=[]
#     for token in tokens:
#         stemTokens.append(stemmer.stem(token))
#     return stemTokens

In [ ]:
# df["Review"]=df["Review"].apply(lambda x: stemText(x))

In [ ]:
df["Review"]

In [ ]:
def generateCleanText(tokens):
    cleanText=" ".join([token for token in tokens])
    return cleanText

In [ ]:
df["Review"]=df["Review"].apply(lambda x: generateCleanText(x))

In [ ]:
df["Review"]

In [ ]:
sid_obj = SentimentIntensityAnalyzer()

In [ ]:
sid_obj.polarity_scores("i like this movie very much. it was a great movie")

In [ ]:
sid_obj.polarity_scores("movie was ok")

In [ ]:
def sentimentAnalysis(sentence):
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict["compound"]

In [ ]:
df["sentiment"]=df["Review"].apply(lambda x: sentimentAnalysis(x))

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
STOP_WORDS

In [ ]:
df_new=df.drop(columns=["Review", "Review Time"])

In [ ]:
df_new

In [ ]:
df_new.to_csv(file_name+"_user_item_map_sentiment.csv", index=False)